In [1]:
import os
import re
import pandas as pd
import spacy

In [2]:
a1_file="/Users/saha/Workspace/PHAEDRA_corpus/test/10099659.a1"
a2_file="/Users/saha/Workspace/PHAEDRA_corpus/test/10099659.a2"
txt_file="/Users/saha/Workspace/PHAEDRA_corpus/test/10099659.txt"

In [3]:
# Read file a1. In this version we ignore entities with discontinuous span 
a1_df=pd.read_csv(a1_file, sep='\t', header=None)
a1_df.columns = ['TermID', 'EntityType', 'Entity']
entities = a1_df[a1_df['TermID'].str.startswith('T')].copy()
entities[['EntityType', 'Start End']] = entities.EntityType.str.split(' ', n=1, expand=True)
continuous_entities = entities[~entities['Start End'].str.contains(';')].copy()
continuous_entities[['Start', 'End']] = continuous_entities['Start End'].str.split(' ', expand=True)
continuous_entities.iloc[0]

TermID                               T1
EntityType    Pharmacological_substance
Entity                   5-fluorouracil
Start End                         27 41
Start                                27
End                                  41
Name: 0, dtype: object

In [8]:
# Read the txt file

entity_map = {"Pharmacological_substance": "PS", "Disorder": "DS", "Subject": "S"}
with open(txt_file, 'r') as rd:
    content = rd.read()
#print(content[0:27])
#print(content[int(continuous_entities.iloc[0]['Start']): int(continuous_entities.iloc[0]['End'])])
#print(continuous_entities.iloc[0:5])
#print(content[int(continuous_entities.iloc[0]['Start']):int(continuous_entities.iloc[0]['End'])])
#pmid = os.path.basename(txt_file)
#print("PMID:" + str(pmid))
#content
nlp = spacy.load("en_core_web_sm")
doc = nlp(content)
entity_idx = 0
entity_flag = 0
#for sent in doc.sents:
#    print("SENTENCE")
#    print(sent)
#    tokens = nlp(sent.text)
#    print("TOKENS")
#print("content:" + content[0:18])
sents = doc.sents

start=0
end=0
i=0
sent_dict = {}
for s in sents:
    print("Senetence" + str(i))
    idx = content.index(s.text)
    print("idx:" + str(idx))
    if i==0:
        end=len(s.text)
    else:
        start = idx
        end = start + len(s.text)
    sent_dict[i] = {'Start': start, 'End': end}
    print(s)
    print("Start:" + str(start) + ", End:" + str(end))
    print("from Content:" + content[start:end +1])
    i=i+1

print(sent_dict)

for t in doc:
    #print(t.text.replace('\n','NEW'))
    #print("Entity Index: " + str(entity_idx) + "\tToken Index:" + str(t.idx))
    if t.idx>=int(continuous_entities.iloc[entity_idx]['Start']):
        if t.idx<int(continuous_entities.iloc[entity_idx]['End']):
            print(t.text + "\t" + entity_map[continuous_entities.iloc[entity_idx]['EntityType']])
            #print(t.text)
            entity_flag = 1
        else:
            if entity_flag==1:
                entity_idx=entity_idx+1
            entity_flag = 0
            print(t.text + "\tO")
    else:
        print(t.text + "\tO")



Senetence0
idx:0
10099659

Prolonged severe 5-fluorouracil-associated neurotoxicity in a patient with dihydropyrimidine dehydrogenase deficiency.


Start:0, End:130
from Content:10099659

Prolonged severe 5-fluorouracil-associated neurotoxicity in a patient with dihydropyrimidine dehydrogenase deficiency.

5
Senetence1
idx:130
5-Fluorouracil (5-FU) is an analogue of pyrimidine nucleosides that is widely used in the treatment of head and neck, breast, ovarian, and colon cancer.
Start:130, End:282
from Content:5-Fluorouracil (5-FU) is an analogue of pyrimidine nucleosides that is widely used in the treatment of head and neck, breast, ovarian, and colon cancer. 
Senetence2
idx:283
Stomatitis, diarrhea, dermatitis, and myelosuppression are the main toxicities of 5-FU.
Start:283, End:370
from Content:Stomatitis, diarrhea, dermatitis, and myelosuppression are the main toxicities of 5-FU. 
Senetence3
idx:371
A less frequent side effect that is becoming more recognized is neurologic toxicity.


In [87]:
import re
# read a2 file
# lines start with T, R, E, A
# different parsing rule for each of these
# Lines beginning with "R" represents relation annotation. It follows following format:
# - A unique id for the relation annotation. By convention, this starts with R, followed by a numerical value.
# - A TAB character.
# - The Relation type label assigned to the annotation. This is either Subject_Disorder, is_equivalent or Coreference.
# Details of the two text spans that are linked in the relation. Each text span that is linked in a relation annotation is referred to as an argument. The first argument is denoted by the label "Arg1" and the second argument is denoted by the label "Arg2". In each case, the argument label is followed by a colon, and then by the ID of the corresponding text span (which corresponds to one of the T annotations introduced above).
#     - is_equivalent relations link two NE spans of the same that refer to the same concept
#     - In Subject_Disorder relations, Arg1 corresponds to the Subject annotation, and Arg2 corresponds to the Disorder.
#     - In Coreference relations, Arg1 corresponds to the anaphoric Coreferring_mention annotation, and Arg2 corresponds to the NE to which the anaphoric expression refers
# Lines beginning with "T" - These correspond to event trigger phrases. They have exactly the same format as the lines in a1 files (see above), except that the semantic label corresponds to an event type.
# Lines beginning with "E" - These lines link together event triggers, event participants and cues for event attribute assignment
# Each such line consists of the the following:
# - A unique id for the event. By convention, this starts with E, followed by a numerical value.
# - A TAB character.
# - The event type, followed by a colon and then the id of the event trigger.
# - Information about event participants and cues for attribute assignment, each separated by a space. For each partiicpant/cue, the format is as follows:
#       - The semantic role label assigned to the participant (or has_cue for attribute cues), followed by a colon
#       - The id of the participant/cue.
#       - For cues, the id will always start with a "T", corresponding the an annotation in the associated a1 files
#       - For participants, the id may start with a "T" if the participant is an NE (the id will correspond to an annotation in the a1 file), or the participant may start with an "E" is the participant is itself an event (which is listed in the same a2 file)
# Lines beginning with "A" - These correspond to event attribute annotations. Note that these will only occur if a non-default value for one or more of the attributes has bee n assigned to the event. The lines have the following format:
#    - A unique id for the attribute annotation. By convention, this starts with A, followed by a numerical value.
#    - A TAB character.
#    - The name of the attribute
#    - The id of the event to which the attribute has been assigned
#    - For the "Manner" attribute ONLY, the non-default value assigned to the attribute (i.e, either "High" or "Low"). For the binary-valued "Speculated" and "Negated" attributes, it is implicit that the value is True, since the annotation is not present if the attribute has the default value of False

def parse_event_trigger(line):
    match = re.match("(T\d+)\t([^ ]+)\s+(\d+)\s+(\d+)\t(.*)$", line)
    if match and len(match.groups())==5:
        return match.group(1), match.group(2), match.group(3), match.group(4), match.group(5)
    return None, None, None, None, None


def parse_relation_annotation(line):
    match = re.match("(R\d+)\t([^ ]+)\s+Arg1:(T\d+)\s+Arg2:(T\d+)$", line)
    if match and len(match.groups())==4:
        return match.group(1), match.group(2), match.group(3), match.group(4)
    return None, None, None, None


def parse_event(line):
    match = re.match("(E\d+)\t(.*)", line.strip())
    if match and len(match.groups())>1:
        event_trigger_link = dict(x.split(":") for x in match.group(2).split(" "))
        return match.group(1), event_trigger_link
    else:
        print('No match')
    return None, None, None, None


def parse_attribute_annotation(line):
    match = re.match("(A\d+)\t([^ ]+)\s+(E\d+)\s?(High|Low)?$", line)
    if match and len(match.groups())==4:
        return match.group(1), match.group(2), match.group(3), match.group(4)
    if match and len(match.groups())==3:
        return match.group(1), match.group(2), match.group(3), None
    return None, None, None, None


with open(a2_file, 'r') as A2:
    while(1):
        line = A2.readline()
        if not line:
            break
        if line.startswith('T'):
            term_id, entity_type, start, end, entity = parse_event_trigger(line)
            print("Event trigger phrase")
            print(term_id + ":" + entity_type + ":" + start + ":" + end + ":" + entity)
        if line.startswith('R'):
            relation_id, relation_type, arg1, arg2 = parse_relation_annotation(line)
            print("Relation Annotation")
            print(relation_id + ":" + relation_type + ":" + arg1 + ":" + arg2)
        if line.startswith('E'):
            event_id, event_trigger_link = parse_event(line)
            print(event_trigger_link)
        if line.startswith('A'):
            attr_annotation, event_type, event_id, measure = parse_attribute_annotation(line)
            print("{}\t{}\t{}\t{}".format(attr_annotation, event_type, event_id, measure))

Event trigger phrase
T35:Adverse_effect:42:52:associated
Event trigger phrase
T42:Potential_therapeutic_effect:220:229:treatment
Event trigger phrase
T43:Adverse_effect:351:361:toxicities
Event trigger phrase
T18:Potential_therapeutic_effect:685:694:receiving
Event trigger phrase
T46:Adverse_effect:855:867:side effects
Event trigger phrase
T48:Adverse_effect:1128:1140:side effects
Event trigger phrase
T58:Adverse_effect:736:743:develop
Event trigger phrase
T4:Adverse_effect:1595:1599:risk
Relation Annotation
R1:Subject_Disorder:T36:T3
Relation Annotation
R2:is_equivalent:T5:T33
Relation Annotation
R5:Subject_Disorder:T15:T19
Relation Annotation
R6:Subject_Disorder:T47:T22
Relation Annotation
R3:Subject_Disorder:T15:T14
{'Adverse_effect': 'T35', 'has_subject': 'T36', 'affects': 'T2', 'has_agent': 'T1', 'has_cue': 'T60'}
{'Potential_therapeutic_effect': 'T42', 'affects': 'T6', 'has_agent': 'T5'}
{'Adverse_effect': 'T43', 'has_agent': 'T12', 'affects': 'T10'}
{'Potential_therapeutic_effec